In [1]:
from flask import Flask, request
from flask_sqlalchemy import SQLAlchemy
from sportsreference.nba.roster import Roster, Player
from sportsreference.nba.player import AbstractPlayer
from sportsreference.nba.teams import Teams

In [2]:
APP = Flask(__name__)
APP.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///nbadb.sqlite3'
APP.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
DB = SQLAlchemy(APP)
DB.init_app(APP)
# engine = create_engine('sqlite:///current.sqlite3')
# meta = MetaData(engine)

In [3]:
DB.drop_all()
DB.metadata.clear()

In [4]:
class NBATeam(DB.Model):
    __tablename__ = "nba_teams"
    __table_args__ = {"schema": "nbadb"}
    abbr = DB.Column(DB.String(4), primary_key=True, unique=True, nullable=False)
    name = DB.Column(DB.String(30), unique=True, nullable=False)
    
    def __repr__(self):
        return '<NBATeam %r>' % self.name

In [5]:
class NBAPlayer(DB.Model):
    __tablename__ = "nba_players"
    __table_args__ = {"schema": "nbadb"}
    ref_id = DB.Column(DB.String(10), primary_key=True, unique=True, nullable=False)
    name = DB.Column(DB.String(30), nullable=False)
    Team = DB.Column(DB.String(4), DB.ForeignKey("nbadb.nba_teams.abbr"), nullable=False)
    FG_Pct = DB.Column(DB.Float, nullable=False)
    TP_Pct = DB.Column(DB.Float, nullable=False)
    FT_Pct = DB.Column(DB.Float, nullable=False)
    Min_per_game = DB.Column(DB.Float, nullable=False)
    Pts_per_game = DB.Column(DB.Float, nullable=False)
    TRB_per_game = DB.Column(DB.Float, nullable=False)
    Asts_per_game = DB.Column(DB.Float, nullable=False)
    WS_per_game = DB.Column(DB.Float, nullable=False)
    BPM = DB.Column(DB.Float, nullable=False)
    VORP = DB.Column(DB.Float, nullable=False)
    
    organization = DB.relationship("NBATeam")
    
    def __repr__(self):
        return '<NBAPlayer %r>' % self.name

In [6]:
teams = Teams()
teamabbs = []
for team in teams:
    teamabbs.append(team.abbreviation)

for abb in teamabbs:
    squad = Roster(abb, slim=True)
    squaddict = squad.players
    squadIDs = list(squaddict.keys())

In [7]:
def add_teams(teams):
    for team in teams:
        nbateam = NBATeam(abbr=team.abbreviation,
                          name=team.name)
        DB.session.add(nbateam)
    DB.session.commit()


In [8]:
def add_players(squadIDs):
    for id in squadIDs:
        player = Player(id)
        nbaplayer = NBAPlayer(ref_id=player.player_id,
                              name=player.name,
                              Team=player.team_abbreviation,
                              FG_Pct=player.field_goal_percentage,
                              TP_Pct=player.three_point_percentage,
                              FT_Pct=player.free_throw_percentage,
                              Min_per_game=(player.minutes_played/player.games_played),
                              Pts_per_game=(player.points/player.games_played),
                              TRB_per_game=(player.total_rebounds/player.games_played),
                              Asts_per_game=(player.assists/player.games_played),
                              WS_per_game=player.win_shares_per_48_minutes,
                              BPM=player.box_plus_minus,
                              VORP=player.value_over_replacement_player
                              )
        DB.session.add(nbaplayer)
    DB.session.commit()

In [9]:
DB.create_all()

add_teams(teams)

OperationalError: (sqlite3.OperationalError) unknown database "nbadb" [SQL: 'PRAGMA "nbadb".table_info("nba_teams")'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
add_players(squadIDs)

In [ ]:
# table1 = Table('nba_teams', meta,
#                DB.Column('abbr', DB.String(4), primary_key=True, unique=True, nullable=False),
#                DB.Column('name', DB.String(30), unique=True, nullable=False))

# table2 = Table('nba_players', meta,
#                DB.Column('ref_id', DB.String(10), primary_key=True, unique=True, nullable=False),
#                DB.Column('name', DB.String(30), nullable=False),
#                DB.Column('Team', DB.String(4), DB.ForeignKey("nbadb.nba_teams.abbr"), nullable=False),
#                DB.Column('FG_Pct', DB.Float, nullable=False), 
#                DB.Column('TP_Pct',DB.Float, nullable=False),
#                DB.Column('FT_Pct', DB.Float, nullable=False),
#                DB.Column('Min_per_game', DB.Float, nullable=False),
#                DB.Column('Pts_per_game', DB.Float, nullable=False),
#                DB.Column('TRB_per_game', DB.Float, nullable=False),
#                DB.Column('Asts_per_game', DB.Float, nullable=False),
#                DB.Column('WS_per_game', DB.Float, nullable=False),
#                DB.Column('BPM', DB.Float, nullable=False),
#                DB.Column('VORP', DB.Float, nullable=False))

# meta.crteate_all()